# COMPUTATIONAL THEORY PROBLEMS

In [100]:
# IMPORTS
import numpy as np

## Problem 1: Binary Words and Operations
SHA-256 uses seven core bitwise functions that operate on 32-bit words. These functions manipulate individual bits using logical operations (```AND, OR, XOR, NOT```) and bit shifts/rotations. All operations must be performed with 32-bit arithmetic to match the SHA-256 specification.

### 1.1 Parity Function - computes the bitwise XOR of three 32-bit words

**How this function works:** It takes three integers and converts them to 32-bit integers (``` np.uint32 ```). It performs ```XOR``` operations which compare bits pairwise, returning 1 when bits are different and 0 when they're the same.

In [101]:
def Parity(x, y, z):
    """
    Bitwise parity function for SHA-256.
    
    Returns the XOR of three 32-bit words: x ⊕ y ⊕ z
    
    For each bit position, returns 1 if an odd number of inputs 
    have a 1 bit at that position.
    
    Args:
        x, y, z: 32-bit unsigned integers
        
    Returns:
        32-bit unsigned integer
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return x ^ y ^ z


In [102]:
# Test Parity function
print("Testing Parity:")

# Test 1: Basic XOR property
print(f"Parity(0xF, 0xF, 0xF) = {Parity(0xF, 0xF, 0xF):#x}")  # Should be 0xF

# Test 2: Cancellation (x ⊕ y ⊕ y = x)
print(f"Parity(0xABCD, 0x1234, 0x1234) = {Parity(0xABCD, 0x1234, 0x1234):#x}")  # Should be 0xABCD

# Test 3: All zeros
print(f"Parity(0x0, 0x0, 0x0) = {Parity(0x0, 0x0, 0x0):#x}")  # Should be 0x0

# Test 4: All ones
print(f"Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) = {Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF):#010x}")  # Should be 0xFFFFFFFF

Testing Parity:
Parity(0xF, 0xF, 0xF) = 0xf
Parity(0xABCD, 0x1234, 0x1234) = 0xabcd
Parity(0x0, 0x0, 0x0) = 0x0
Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) = 0xffffffff


### 1.2 Ch (Choose) Function - Select bits from y or z based on x

**How this function works:** The idea here is the same as in the Parity function (see 1.1) but instead of only doing ```XOR``` operations we first do an ```AND``` operation between x and y, then we do an ```AND``` operation between the ```NOT``` of x and z. Finally, we ```XOR``` these two results together. The ```AND``` operations act as filters that select which bits to keep from y and z, while the ```XOR``` combines them into the final result. 

```(x & y)``` is just a basic ```AND``` operation (if both bits are 1 it returns 1). 

```(~x & z)``` flips all bits in x before carrying out the ```AND``` operation with z.

We then XOR the result of these two operations.

In [103]:
def Ch(x, y, z):
    """
    Bitwise choice function for SHA-256.
    
    For each bit position, returns the bit from y if the corresponding
    bit in x is 1, otherwise returns the bit from z

    Args:
        x, y, z: 32-bit unsigned integers

    Returns:
        32-bit unsigned integer result of the bitwise choice operation
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return (x & y) ^ (~x & z)

In [104]:
# Test Ch function
print("Testing Ch:")

# Test 1: All 1s in x selects -> y entirely
print(f"Ch(0xFFFFFFFF, 0xAAAAAAAA, 0x55555555) = {Ch(0xFFFFFFFF, 0xAAAAAAAA, 0x55555555): }")  # Should be 0xAAAAAAAA

# Test 2: All 0s in x selects -> z entirely
print(f"Ch(0x00000000, 0xAAAAAAAA, 0x55555555) = {Ch(0x00000000, 0xAAAAAAAA, 0x55555555):#010x}")  # Should be 0x55555555

# Test 3: Mixed selector - alternates between y and z
print(f"Ch(0xF0F0F0F0, 0xFFFFFFFF, 0x00000000) = {Ch(0xF0F0F0F0, 0xFFFFFFFF, 0x00000000):#010x}")  # Should be 0xF0F0F0F0

Testing Ch:
Ch(0xFFFFFFFF, 0xAAAAAAAA, 0x55555555) =  2863311530
Ch(0x00000000, 0xAAAAAAAA, 0x55555555) = 0x55555555
Ch(0xF0F0F0F0, 0xFFFFFFFF, 0x00000000) = 0xf0f0f0f0


### 1.3 Maj (Majority) Function - returns the value that appears in at least 2 out of 3 inputs

**How this function works:** We perform three ```AND``` operations to find where pairs of inputs are the same (`x & y`, `x & z`, `y & z`), then ```XOR``` these results together. This produces 1 at positions where at least two inputs have 1.

In [105]:
def Maj(x, y, z):
    """
    Majority function for SHA-256.
    
    Maj(x, y, z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    
    For each bit position, returns the bit value that appears 
    in at least 2 of the 3 inputs (the majority).
    
    Args:
        x, y, z: 32-bit unsigned integers
        
    Returns:
        32-bit unsigned integer
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return (x & y) ^ (x & z) ^ (y & z)

In [106]:
# Test Maj function
print("Testing Maj:")

# Test 1: Two inputs the same
print(f"Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000) = {Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000):#010x}")  # Should be 0xFFFFFFFF (majority is 1)

# Test 2: Two inputs the same (other pattern)
print(f"Maj(0x00000000, 0xFFFFFFFF, 0xFFFFFFFF) = {Maj(0x00000000, 0xFFFFFFFF, 0xFFFFFFFF):#010x}")  # Should be 0xFFFFFFFF

# Test 3: All same
print(f"Maj(0xAAAAAAAA, 0xAAAAAAAA, 0xAAAAAAAA) = {Maj(0xAAAAAAAA, 0xAAAAAAAA, 0xAAAAAAAA):#010x}")  # Should be 0xAAAAAAAA

Testing Maj:
Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000) = 0xffffffff
Maj(0x00000000, 0xFFFFFFFF, 0xFFFFFFFF) = 0xffffffff
Maj(0xAAAAAAAA, 0xAAAAAAAA, 0xAAAAAAAA) = 0xaaaaaaaa


## Problem 2: Fractional Parts of Cube Roots

In [107]:
# Problem 2

## Problem 3: Padding

In [108]:
# Problem 3

## Problem 4: Hashes

In [109]:
# Problem 4

## Problem 5: Passwords

In [110]:
# Problem 5

# END